In [1]:
#imports
import numpy as np, pandas as pd, os, openpyxl, re

In [2]:
with open("exp2_parovr.txt","r") as f:
    txt = list(filter(lambda x:not re.match(r"\s*$",x), f.read().split("\n-------end session-------\n")))

Runtime was capped at 2000ms and 5 trials of every setting done and averaged out.

In [3]:
#initializing each necessary dataframe
df = pd.DataFrame(columns=['Hash Table Type','Mostly Reads','Heavy Writes'])
df.loc[:,'Hash Table Type'] = ['Serial', 'Striped Lock', 'Optimistic Striped Lock']
df.set_index('Hash Table Type',inplace=True)
df.loc[:,:] = np.zeros((3,2))

In [4]:
htypedict = {0:'Serial', 1:'Striped Lock', 2:'Optimistic Striped Lock'}
optypedict = {.09: 'Mostly Reads', .45: 'Heavy Writes'}

In [5]:
#parse, then avg
for s in range(len(txt)):
    processed = int(re.search(r"(?<=processed: )[0-9]+",txt[s]).group())
    hashtype = int(re.search(r"(?<=htype: )[0-9]+",txt[s]).group())
    addpor = float(re.search(r"(?<=addpor: )[0-9]*\.[0-9]+",txt[s]).group())
    df.loc[htypedict[hashtype],optypedict[addpor]] += processed
df /= (5*2000)

In [6]:
#turn into avg throughput ratio to serial, write to excel
with pd.ExcelWriter('expdata.xlsx') as writer:
    df.to_excel(writer,sheet_name="exp2")

In [7]:
df

,Mostly Reads,Heavy Writes
Hash Table Type,,
Serial,149.8975,128.5182
Striped Lock,126.4246,120.9476
Optimistic Striped Lock,129.5041,120.3621


In [8]:
with open("exp3_spdup.txt","r") as f:
    txt = list(filter(lambda x:not re.match(r"\s*$",x), f.read().split("\n-------end session-------\n")))

In [9]:
#initializing each necessary dataframe
df_rd5 = pd.DataFrame(columns=['nthreads','Serial','Striped Lock','Optimistic Striped Lock'])
df_rd5.loc[:,'nthreads'] = [1,2,4,8,16,32]
df_rd5.set_index('nthreads',inplace=True)
df_rd5.loc[:,:] = np.zeros((6,3))
df_rd75, df_rd9, df_rd99, df_wr5, df_wr75, df_wr9, df_wr99 = [df_rd5.copy() for _ in range(7)]

In [10]:
htypedict = {0:'Serial', 1:'Striped Lock', 2:'Optimistic Striped Lock'}
loaddict = {(.09,.5):df_rd5,(.09,.75):df_rd75,(.09,.9):df_rd9,(.09,.99):df_rd99,\
            (.45,.5):df_wr5,(.45,.75):df_wr75,(.45,.9):df_wr9,(.45,.99):df_wr99}

In [11]:
#parse, then avg
for s in range(len(txt)):
    processed = int(re.search(r"(?<=processed: )[0-9]+",txt[s]).group())
    hashtype = int(re.search(r"(?<=htype: )[0-9]+",txt[s]).group())
    addpor = float(re.search(r"(?<=addpor: )[0-9]*\.[0-9]+",txt[s]).group())
    hitrate = float(re.search(r"(?<=hitrate: )[0-9]*\.[0-9]+",txt[s]).group())
    nthreads = int(re.search(r"(?<=nsrc: )[0-9]+",txt[s]).group())
    loaddict[(addpor,hitrate)].loc[nthreads,htypedict[hashtype]] += processed
for key in loaddict.keys():
    loaddict[key] /= 5*2000
    for i in range(1,3):
        loaddict[key].iloc[:,i] /= loaddict[key].iloc[:,0] 

In [12]:
#turn into avg throughput ratio to serial, write to excel
with pd.ExcelWriter('expdata.xlsx',mode='a') as writer:
    for key in loaddict.keys():
        loaddict[key].to_excel(writer,sheet_name="exp3 {}".format(key))

In [13]:
for key in loaddict.keys():
    print(loaddict[key])

            Serial  Striped Lock  Optimistic Striped Lock
nthreads                                                 
1         157.4762      1.002607                 1.006466
2         173.5764      1.742806                 1.654586
4         163.8252      3.578836                 2.872871
8         150.5392      7.047155                 3.717569
16        138.2413      5.307995                 4.659982
32        139.6563      0.161223                 5.981242
            Serial  Striped Lock  Optimistic Striped Lock
nthreads                                                 
1         168.8801      0.935173                 0.938970
2         164.1576      1.847769                 1.764169
4         150.7986      3.901201                 3.210244
8         144.1533      7.181296                 4.117657
16        137.6144      5.505602                 5.184822
32        119.1748      0.233591                 3.508310
            Serial  Striped Lock  Optimistic Striped Lock
nthreads      

In [14]:
with open("optovr.txt","r") as f:
    txt = list(filter(lambda x:not re.match(r"\s*$",x), f.read().split("\n-------end session-------\n")))

In [15]:
#initializing each necessary dataframe
df = pd.DataFrame(columns=['Hash Table Type','Mostly Reads','Heavy Writes'])
df.loc[:,'Hash Table Type'] = ['Serial', 'Striped Lock', 'Optimistic Striped Lock']
df.set_index('Hash Table Type',inplace=True)
df.loc[:,:] = np.zeros((3,2))

In [16]:
htypedict = {0:'Serial', 1:'Striped Lock', 2:'Optimistic Striped Lock'}
optypedict = {.09: 'Mostly Reads', .45: 'Heavy Writes'}

In [17]:
#parse, then avg
for s in range(len(txt)):
    processed = int(re.search(r"(?<=processed: )[0-9]+",txt[s]).group())
    hashtype = int(re.search(r"(?<=htype: )[0-9]+",txt[s]).group())
    addpor = float(re.search(r"(?<=addpor: )[0-9]*\.[0-9]+",txt[s]).group())
    df.loc[htypedict[hashtype],optypedict[addpor]] += processed
df /= (5*2000)

In [18]:
#turn into avg throughput ratio to serial, write to excel
with pd.ExcelWriter('expdata.xlsx',mode='a') as writer:
    df.to_excel(writer,sheet_name="optovr")

In [19]:
df

,Mostly Reads,Heavy Writes
Hash Table Type,,
Serial,166.4638,127.6528
Striped Lock,820.1316,803.0515
Optimistic Striped Lock,605.2175,280.4652
